In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json

from sklearn import svm

# we create 70 separable points
np.random.seed(0)
X = np.r_[np.random.randn(35, 2) - [2, 2], np.random.randn(35, 2) + [2, 2]]
Y = [0] * 35 + [1] * 35

# figure number
fignum = 1

upper_soft_boundry = None
lower_soft_boundry = None
hard_boundry = None
boundry_x = None
support_vectors = None

# fit the model
for name, penalty in (("unreg", 1), ("reg", 0.05)):
    clf = svm.SVC(kernel="linear", C=penalty)
    clf.fit(X, Y)

    # get the separating hyperplane
    w = clf.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(-5, 5)
    yy = a * xx - (clf.intercept_[0]) / w[1]

    # plot the parallels to the separating hyperplane that pass through the
    # support vectors (margin away from hyperplane in direction
    # perpendicular to hyperplane). This is sqrt(1+a^2) away vertically in
    # 2-d.
    margin = 1 / np.sqrt(np.sum(clf.coef_**2))
    yy_down = yy - np.sqrt(1 + a**2) * margin
    yy_up = yy + np.sqrt(1 + a**2) * margin

    # plot the line, the points, and the nearest vectors to the plane
    plt.figure(fignum, figsize=(4, 3))
    plt.clf()
    plt.plot(xx, yy, "k-") # margin
    plt.plot(xx, yy_down, "k--") # soft_margin
    plt.plot(xx, yy_up, "k--") # soft_margin

    plt.scatter(
        clf.support_vectors_[:, 0],
        clf.support_vectors_[:, 1],
        s=80,
        facecolors="none",
        zorder=10,
        edgecolors="k",
        cmap=plt.get_cmap("RdBu"),
    )
    plt.scatter(
        X[:, 0], X[:, 1], c=Y, zorder=10, cmap=plt.get_cmap("RdBu"), edgecolors="k"
    )

    # naming variables for building .json
    support_vectors = clf.support_vectors_
    boundry_x = xx
    upper_soft_boundry = yy_up
    lower_soft_boundry = yy_down
    hard_boundry = yy

    plt.axis("tight")
    x_min = -4.8
    x_max = 4.2
    y_min = -6
    y_max = 6

    YY, XX = np.meshgrid(yy, xx)
    xy = np.vstack([XX.ravel(), YY.ravel()]).T
    Z = clf.decision_function(xy).reshape(XX.shape)

    # Put the result into a contour plot
    plt.contourf(XX, YY, Z, cmap=plt.get_cmap("RdBu"), alpha=0.5, linestyles=["-"])

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.xticks(())
    plt.yticks(())
    fignum = fignum + 1
    break

plt.show()

model_data = {"points":[], "hard_Bdy": [], "upper_soft_Bdy": [], "lower_soft_Bdy": [], "SV": []}
SV_hash = []

# we get all the Sv from the data for verification
for i in range(0,len(support_vectors)):
  SV_hash.append({"x" : support_vectors[i][0], "y": support_vectors[i][1]})
  
# we get all the points from the data + verify the SV
for i in range(0,len(X)):
  model_data["points"].append({"x" : X[i][0], "y": X[i][1], "group": Y[i], "isSV": {"x" : X[i][0], "y": X[i][1]} in SV_hash})

# we get the two end point of the hard boundry points
model_data["hard_Bdy"].append(
    {
        "x1": boundry_x[0],
        "x2": boundry_x[-1],
        "y1": hard_boundry[0],
        "y2": hard_boundry[-1]
    }
)

# we get the two end point of the soft upper boundry points
model_data["upper_soft_Bdy"].append(
    {
        "x1": boundry_x[0],
        "x2": boundry_x[-1],
        "y1": upper_soft_boundry[0],
        "y2": upper_soft_boundry[-1]
    }
)

# we get the two end point of the soft lower boundry points
model_data["lower_soft_Bdy"].append(
    {
        "x1": boundry_x[0],
        "x2": boundry_x[-1],
        "y1": lower_soft_boundry[0],
        "y2": lower_soft_boundry[-1]
    }
)


with open('model_data_3D_SVM.json', 'w') as json_file:
    json.dump(model_data, json_file, indent=4)